# How to Create a RAG Jupyter Notebook with LangChain

**pip install langchain openai transformers faiss-cpu**

**pip install langchain-community langchain-core**

Source:

Gary Svenson, How to Create a RAG Jupyter Notebook with LangChain,
https://medium.com/towards-agi/how-to-create-a-rag-jupyter-notebook-with-langchain-e5f48c2e4ecf

In [25]:
!pip install numpy pandas langchain langchain_community faiss-cpu openai

In [26]:
import langchain
langchain.__version__


'0.3.14'

In [27]:
import faiss

In [29]:
import os
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

### Loading Text Data

In [30]:
loader = TextLoader('/paris_texas_sites.txt')

documents = loader.load()

In [31]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(documents)

### Creating Text Embeddings

In [32]:
# Paste your API key here. Remember to not share publicly
openai_api_key = open("/openai_mjack.apikey", "r").read().strip()

os.environ["OPENAI_API_KEY"] = openai_api_key

In [33]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.7 MB/s eta 0:00:00


In [34]:
vectorstore = FAISS.from_documents(documents = all_splits, embedding = OpenAIEmbeddings())

### Setting Up the Retrieval Mechanism

In [35]:
faiss_retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

### Choosing a Language Model

In [36]:
llm = OpenAI(temperature=0.7, api_key=openai_api_key)

<ipython-input-36-b878908297b4>:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0.7, api_key=openai_api_key)


### Implementing the RAG: Constructing the RetrievalQA Chain

Source:
https://python.langchain.com/v0.2/docs/versions/migrating_chains/retrieval_qa/

In [37]:
from langchain import hub
from langchain.chains import RetrievalQA

In [38]:
# See full prompt at https://smith.langchain.com/hub/rlm/rag-prompt
prompt = hub.pull("rlm/rag-prompt")

qa_chain = RetrievalQA.from_llm(
    llm, retriever=faiss_retriever, prompt=prompt
)

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [39]:
qa_chain("What are most popular sites in Paris?")

<ipython-input-39-cb39d1db0ec8>:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  qa_chain("What are most popular sites in Paris?")


{'query': 'What are most popular sites in Paris?',
 'result': ' Some of the most popular sites in Paris are the Eiffel Tower, Notre Dame Cathedral, Louvre Museum, and Arc de Triomphe.'}

### Evaluating and Tuning the System

It is crucial to evaluate the generated outputs from the RAG system continually.

Based on the effectiveness and relevance of responses, you might want to adjust some parameters:

- Adjusting Temperature and Nucleus Sampling

- Fine-Tuning Documents

- Monitor Query Performance

### Exploring Advanced Features

#### Adding Contextual Memory

In [40]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory()

<ipython-input-40-61e7e3748c65>:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


#### Other Steps:

- Integrating with APIs and Other Data Sources

- Building Conversation Flows

- Utilizing Custom Callbacks